Extraction

In [ ]:
import os
os.chdir('C:\\Users\\badjo\\OneDrive\\Documents\\CP322FinalProj')
import sys
sys.path.insert(0, 'src')
import utils
csv_path = "data/spanish/spanish_tweets.csv"
df = utils.extract_csv(csv_path)

C:\Users\badjo\OneDrive\Documents\CP322FinalProj
             user                                               text  \
0      @erreborda               termine bien abrumado después de hoy   
1    @shpiderduck                                 me siento abrumado   
2     @Alex_R_art  Me siento un poco abrumado por la cantidad de ...   
3   @anggelinaa97  Salvador la única persona que no la ha abrumad...   
4  @diegoreyesvqz     Denme un helado o algo que ando full abrumado.   

                         date      emotion sentiment  
0   Jan 6, 2024 · 2:53 AM UTC  overwhelmed    scared  
1   Jan 6, 2024 · 2:35 AM UTC  overwhelmed    scared  
2  Jan 6, 2024 · 12:20 AM UTC  overwhelmed    scared  
3  Jan 5, 2024 · 10:38 PM UTC  overwhelmed    scared  
4   Jan 5, 2024 · 8:38 PM UTC  overwhelmed    scared  


Cleaning data

In [ ]:
import pandas as pd
import numpy as np
# removing usernames
df["clean_text"] = df["text"].str.replace(r"@\w+", "", regex=True).str.strip()
# removing urls
df["clean_text"] = df["clean_text"].str.replace(r"http\S+", "", regex=True)
# removing empty texts
df = df[df["clean_text"].str.len() > 0]
print(df["clean_text"].head())


             user                                               text  \
0      @erreborda               termine bien abrumado después de hoy   
1    @shpiderduck                                 me siento abrumado   
2     @Alex_R_art  Me siento un poco abrumado por la cantidad de ...   
3   @anggelinaa97  Salvador la única persona que no la ha abrumad...   
4  @diegoreyesvqz     Denme un helado o algo que ando full abrumado.   

                         date      emotion sentiment  \
0   Jan 6, 2024 · 2:53 AM UTC  overwhelmed    scared   
1   Jan 6, 2024 · 2:35 AM UTC  overwhelmed    scared   
2  Jan 6, 2024 · 12:20 AM UTC  overwhelmed    scared   
3  Jan 5, 2024 · 10:38 PM UTC  overwhelmed    scared   
4   Jan 5, 2024 · 8:38 PM UTC  overwhelmed    scared   

                                          clean_text  
0               termine bien abrumado después de hoy  
1                                 me siento abrumado  
2  Me siento un poco abrumado por la cantidad de ...  
3  Salvado

In [ ]:
from transformers import pipeline, AutoTokenizer
import torch
unmasker = pipeline('fill-mask', model='bert-base-multilingual-cased')
unmasker("Hello I'm a [MASK] model.") 


